In [1]:
from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/1.url'):
    shutil.rmtree('./file/1.url')

os.makedirs('./file/1.url')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
# option.add_argument('--headless=new')
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument(get_header(method='selenium'))
option.add_argument(get_proxy(method='selenium'))
option.page_load_strategy = 'eager'

browser = Chrome(service=Service('../chromedriver'), options=option)
browser.maximize_window()

# = = = = = = = = = = = = = = =

b = 0
while True:
    b += 1

    try:
        browser.get('https://www.amazon.com/?language=en_US')
        
        break
    except KeyboardInterrupt:
        browser.quit()
        break
    except:
        browser.quit()
        continue

# = = = = = = = = = = = = = = =

input(f'''fuck?: ''')

# = = = = = = = = = = = = = = =

for a in range(len(input_)):

    # = = = = = = = = = = = = = = =

    crawler_status = 'error'

    # = = = = = = = = = = = = = = =

    try:
        url = input_.loc[a, 'Url']
        
        dict_param = {}
        list_param = url.split('?')[1].split('&') if '?' in url else []
        for param in list_param:
            dict_param[param.split('=')[0]] = param.split('=')[1]
        
        dict_param.pop('pg', '')
        dict_param['language'] = 'en_US'
        
        url = url.split('?')[0] + '?'
        for key, value in dict_param.items():
            url += f'{key}={value}&'
        
        list_url, page = [], 1
        while True:
            b = 0
            while True:
                b += 1

                try:
                    request_url = f'{url}pg={page}'
                    
                    browser.get(request_url)

                    soup = BeautifulSoup(browser.page_source, 'lxml')
                    html = etree.HTML(str(soup))

                    list_dict = json.loads(html.xpath('//div[@data-client-recs-list]/@data-client-recs-list')[0])
        
                    if list_dict:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue
        
            # = = = = = = = = = = = = = = =

            list_url += [f'''https://www.amazon.com/dp/{dict_['id']}?language=en_US''' for dict_ in list_dict]
        
            # = = = = = = = = = = = = = = =

            if a % 100 == 0 and a != 0:
                clear_output()

            print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{request_url} > Page {page}\n[剩余数量：{len(input_) - a - 1}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

            # = = = = = = = = = = = = = = =

            page += 1
        
            # = = = = = = = = = = = = = = =

            if html.xpath('//li[@class="a-disabled a-last"]'):
                break
        
        # = = = = = = = = = = = = = = =

        if not list_url:
            raise

        # = = = = = = = = = = = = = = =

        output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                               'Url': list_url})
        
        output.to_excel(f'''./file/1.url/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

        # = = = = = = = = = = = = = = =

        crawler_status = 'ok'
    except:
        df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
        df_temp.loc[0, 'Request_Url'] = request_url

        output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
        
# = = = = = = = = = = = = = = =

browser.quit()

# = = = = = = = = = = = = = = =

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/1.url/{file}',
              f'''./file/1.url/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2



fuck?:  fuck


[尝试次数：1] - 1.https://www.amazon.com/gp/bestsellers/automotive/15730511/ref=pd_zg_hrsr_automotive?language=en_US&pg=1 > Page 1
[剩余数量：1] - [当前时间：11:08:19]

[尝试次数：1] - 1.https://www.amazon.com/gp/bestsellers/automotive/15730511/ref=pd_zg_hrsr_automotive?language=en_US&pg=2 > Page 2
[剩余数量：1] - [当前时间：11:08:21]

[尝试次数：1] - 2.https://www.amazon.com/gp/bestsellers/wireless/7072562011/ref=pd_zg_hrsr_wireless?language=en_US&pg=1 > Page 1
[剩余数量：0] - [当前时间：11:08:39]

[尝试次数：1] - 2.https://www.amazon.com/gp/bestsellers/wireless/7072562011/ref=pd_zg_hrsr_wireless?language=en_US&pg=2 > Page 2
[剩余数量：0] - [当前时间：11:08:40]

输出ing...



Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 2853.27it/s]

Done ~
